# Inven 롤 선수 공식 대회 전적 크롤링
 - inven사이트에서 한국 롤 선수들의 공식 대회 전적 (KDA, 킬관여율)을 크롤링

## Selenium crawler

#### 참조한 사이트 : 
- https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/  ## Selenium 전체적인 사용법
- http://yumere.tistory.com/75  ## submit 함수 사용법

## Inven crawling

#### 참조한 사이트
- https://www.guru99.com/xpath-selenium.html  ## xpath에 관한 글

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
##chrome automation
driver = webdriver.Chrome('/Users/김호준/Documents/chrome_webdriver/chromedriver') ##install chromedriver in 
                                                                      ##your computer and locate its directory.
driver.implicitly_wait(3) 

In [3]:
## open LOL inven site
driver.get('http://lol.inven.co.kr/dataninfo/match/playerList.php?iskin=lol&shipGroup=&shipCode=&playerName=&champ=0&targetName=&startDate=&endDate=')

In [4]:
## find '더보기' element's javascript and click it
import time
while(True):
    more_data = driver.find_element_by_xpath('//*[@class="moreline"]//a[@href="javascript:void(0);"]') ## '더보기' button javascript
    more_data.click()
    time.sleep(10) ##give loading time to chrome browser

KeyboardInterrupt: 

## Use BeautifulSoup

In [5]:
## get html source of current page
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Load .csv file that saved by selenium module
prettify = soup('tbody')[1].prettify('euc-kr')
with open("output_score.html", "wb") as file:
    file.write(prettify)

In [7]:
file = open("output_score.html", 'r')
html = BeautifulSoup(file, 'html.parser')
data = html.find_all("td")

In [8]:
print(data[0:11]) #0 : index, 1 : competition name, 2 : player name, 3 : played character, 4 : summoner spell, 5 : win/lose, 6 : Kill, 
                  #7 : Death, 8 : Assist, 9 : KDA, 10 : kill contribution

[<td>
   17.12.11
  </td>, <td class="left" onmouseout="LOL.CommonLayer.hide();" onmouseover="LOL.CommonLayer.showText('All-Star 2017 결승전 5세트');">
   2017 올스타
  </td>, <td class="left">
<a href="http://lol.inven.co.kr/dataninfo/proteam/progamer.php?code=998">
    LMS Ziv
   </a>
</td>, <td>
<div class="none">
    마오카이
   </div>
<img onmouseout="LOL.CommonLayer.hide();" onmouseover="LOL.CommonLayer.showText('마오카이');" src="http://static.inven.co.kr/image_2011/site_image/lol/dataninfo/icon/champions/Maokai_Square_0_2.jpg"/>
</td>, <td>
<img onmouseout="LOL.CommonLayer.hide();" onmouseover="LOL.CommonLayer.showText('점멸');" src="http://static.inven.co.kr/image_2011/site_image/lol/dataninfo/icon/spells/lol_spell_flash.png"/>
<img onmouseout="LOL.CommonLayer.hide();" onmouseover="LOL.CommonLayer.showText('순간이동');" src="http://static.inven.co.kr/image_2011/site_image/lol/dataninfo/icon/spells/lol_spell_teleport.png"/>
</td>, <td>
   패
  </td>, <td>
   0
  </td>, <td>
   5
  </td>, <td>
   6
  

In [9]:
# Parsing data 
str_data = []
for i in range(0, len(data)):
    string = str(data[i])
    data[i] = str(data[i])
    if(i % 11 == 0):
        str_data.append(string.replace("<td>\n   ", '').replace("\n  </td>", ''))
    elif(i % 11 == 1):
        begin = string.find("'")
        end = string.find("'", begin+1)
        str_data.append(string[begin+1:end])
    elif(i % 11 == 2):
        begin = string.find('">\n   ')
        end = string.find("\n", begin+7)
        str_data.append(string[begin+6:end])
    elif(i % 11 == 3):
        begin = string.find("'")
        end = string.find("'", begin+1)
        str_data.append(string[begin+1:end])
    elif(i % 11 == 4):
        begin1 = string.find("'")
        end1 = string.find("'", begin1+1)
        str_data.append(string[begin1+1:end1])
        
        begin2 = string.find("'", end1+1)
        end2 = string.find("'", begin2+1)
        str_data.append(string[begin2+1:end2])
    elif(i % 11 == 5):
        str_data.append(string[8])
    elif(i % 11 == 6):
        str_data.append(string[8])
    elif(i % 11 == 7):
        str_data.append(string[8])
    elif(i % 11 == 8):
        str_data.append(string[8])
    elif(i % 11 == 9):
        end = string.find("\n  </td>")
        str_data.append(string[8:end])
    elif(i % 11 == 10):
        end = string.find("\n  </td>")
        str_data.append(string[8:end])

## Move data to Pandas

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Make 2 dimensional array that holds competition data
before_pandas = []
for i in range(0, int(len(str_data)/12)):
    before_pandas.append(str_data[12 * i : 12 * i + 12])

In [12]:
# Naming columns
column_name = ['대회날짜', '대회명', '선수이름', '챔피언', '스펠1', '스펠2', '승/패', '킬', '데스', '어시', 'KDA', '킬관여율']
df = pd.DataFrame(before_pandas, columns=column_name)
df.head(5)

,대회날짜,대회명,선수이름,챔피언,스펠1,스펠2,승/패,킬,데스,어시,KDA,킬관여율
0,17.12.11,All-Star 2017 결승전 5세트,LMS Ziv,마오카이,점멸,순간이동,패,0,5,6,1.2,50%
1,17.12.11,All-Star 2017 결승전 5세트,LMS Karsa,카직스,강타,점멸,패,5,6,3,1.3,67%
2,17.12.11,All-Star 2017 결승전 5세트,LMS FoFo,아지르,점멸,회복,패,5,3,4,3,75%
3,17.12.11,All-Star 2017 결승전 5세트,LMS Bebe,진,점멸,회복,패,1,2,6,3.5,58%
4,17.12.11,All-Star 2017 결승전 5세트,LMS SwordArt,탐 켄치,탈진,점멸,패,1,0,5,7.2,50%


In [13]:
# Make csv of data
df.to_csv("대회전적.csv")